# 최종 35개 feature에서 knn으로 결측치 처리

## 전처리

#### 각 행 결측치 비율 확인
- 6291개의 행 (비율상 전체 train_set에서 2.45%)이 15개의 결측치를 포함하고 있음. 
    -   ##1 .해당 행들 삭제 
    -   ##2. column "배아 해동 경과일"이 결측치 비율이 84.25% / 이에 따라 해당 column 삭제
    

In [87]:
import pandas as pd
import numpy as np


from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

### 35개 feature들 origin train data에서 추출

In [84]:
train3_columns = pd.read_csv("data/train3_updated.csv").drop("ID", axis=1).columns
test3_columns = pd.read_csv("data/test3_updated.csv").drop("ID", axis=1).columns

train4_DF = pd.read_csv("data/train.csv")[train3_columns]
test4_DF = pd.read_csv("data/test.csv")[test3_columns]

#### categorical data & numerical data 결측치 표현 np.nan으로 통일 및 84%이상인 columne들 drop

- 아래는 drop_column: nan ratio 
    - 배란 유도 유형     99.999220
    - 난자 기증자 나이    94.550441
    - 정자 기증자 나이    89.922801
    - 배아 해동 경과일    84.252451


- 아래는 범주형 데이터에서 결측치 표현 모음
    - 시술 당시 나이: 알 수 없음,
    - 특정 시술 유형 : Unknown, nan
    - 배란 유도 유형: 기록되지 않은 시행, 알 수 없음
    - 난자 기증자 나이 : 알 수 없음
    - 정자 기증자 나이 : 알 수 없음


In [ ]:
## categorical data 결측치 np.nan으로 대치
categorical_nan_list = ["알 수 없음", "Unknown", "nan", "기록되지 않은 시행"]
train4_DF.replace(categorical_nan_list, np.nan, inplace=True)
test4_DF.replace(categorical_nan_list, np.nan, inplace=True)


## numerical data 결측치 전부 np.nan으로 대치
train4_DF.replace('', np.nan, inplace=True)
test4_DF.replace('', np.nan, inplace=True)

# 각column의 결측치 비율 계산 
nan_ratio = train4_DF.isnull().mean() * 100

# 결측치 비율이 80프로 이상인 것들 찾기
columns_above_84 = nan_ratio[nan_ratio >= 84].index
columns_above_84 = list(columns_above_84)

train4_DF = train4_DF.drop(columns_above_84, axis=1)
test4_DF = test4_DF.drop(columns_above_84, axis=1)

/var/folders/1b/phqsx_057lg8x6fbhtwvv5w40000gn/T/ipykernel_3629/3902721074.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test4_DF.replace(categorical_nan_list, np.nan, inplace=True)


#### 각 행의 결측치 갯수 확인

#### 결측치 갯수가 너무 많은 행들 drop 
#### 해당 전처리는 train_data에 대해서만 수행한다.
#### 모델 성능을 보고 해당 작업은 추후에 배제될 수 있다.

- 6291개의 data point가 34개의 feature중 15개의 Feaure에 대한 값이 nan 이었다. 
- 이는 다른 data point들이 가지고 있는 결측치의 갯수보다 현저히 많기 때문에 (0,1,2,3) drop

In [86]:
null_num_row = train4_DF.isnull().sum(axis=1)  ##각 행의 결측치 갯수
nan15_filtered_DF = train4_DF[train4_DF.isnull().sum(axis=1) >= 15]

# nan15_filtered_DF에 포함된 행의 index추출
nan15_filtered_indices = nan15_filtered_DF.index

# train4_DF에서 해당 index들 삭제
train4_DF = train4_DF.drop(nan15_filtered_indices)

# trian4_DF인덱스 다시 처음부터 reindexing
train4_DF = train4_DF.reset_index(drop=True)

#### knn-imputing을 위한 label encoding & knn imputing 

- categorical column
    - 이후 catboost에 넘겨서 catboost model 학습 과정에서 내부적으로 처리하게끔 함
    - catboost는 내부적으로 categorical data의 nan값을 하나의 다른 범주로 인식해서 결측치를 처리한다.

- numerical columns
    - categorical column을 제외하고 나머지 numerical columns끼리 knn-imputing을 이용해서 결측치 값을 예측


#### Wrok Flow
1. numerical dtype column들을 모아서 따로 떼어냄
2. standard scaling 후 knn-imputing  (1시간 정도 소모)
3. scaling 이전 값으로 revert
4. 기존 데이터셋의 numerical dtype column 값들을 knn-imputing한 값들로 대체 
5. dataset 저장

In [69]:
# 0.  imputing 대상 칼럼 들만 모아서 떼어내기
train_columns = train4_DF

categorical_columns = []
numerical_columns = []    ##imputing대상 columne

for col in train_columns:
    if train4_DF[col].dtype == object:
        categorical_columns.append(col)
    else:
        numerical_columns.append(col)

numerical_columns.remove("임신 성공 여부")


train_impute_DF = train4_DF[numerical_columns]
test_impute_DF = test4_DF[numerical_columns]


# 1. boolean value들 가지고 오기 / inverse_tranform 이후에 해당 column들에 해당하는 dataset들은 다시 0과 1로 변환 해주기 
binary_columns = []
for col in numerical_columns:
    # 해당 컬럼의 고유값을 집합으로 변환
    unique_values = set(train4_DF[col].unique())  # NaN 제외
    # 고유값이 {0, 1}과 정확히 일치하면 이진 컬럼으로 판단
    if unique_values == {0, 1}:
        binary_columns.append(col)



# 2. standard scaling
scaler = StandardScaler()
train_impute_DF = scaler.fit_transform(train_impute_DF)
test_impute_DF = scaler.fit_transform(test_impute_DF)

# 3. knn-imputing
imputer = KNNImputer(n_neighbors=4)
train_impute_DF = imputer.fit_transform(train_impute_DF)
test_impute_DF = imputer.fit_transform(test_impute_DF)

# 4. revert
train_impute_DF = scaler.inverse_transform(train_impute_DF)
test_impute_DF = scaler.inverse_transform(test_impute_DF)

# 5. PandasDF로 변환 & boolean값들 다시 변환
train_impute_DF = pd.DataFrame(train_impute_DF, columns=numerical_columns)
test_impute_DF = pd.DataFrame(test_impute_DF, columns=numerical_columns)
for col in binary_columns:
    train_impute_DF[col] = train_impute_DF[col].round().astype(int)
    test_impute_DF[col] = test_impute_DF[col].round().astype(int)

#### 기존 train data와 test data의 numerical data를 전처리 완료한 numerical data의 값들로 대치 하고 csv파일로 저장.

In [70]:
for col in train_impute_DF.columns:
    print(f"{train_impute_DF[col].unique()}\n\n")

[1 0]


[0 1]


[0 1]


[0 1]


[1 0]


[1 0]


[0 1]


[4.00297910e+00 1.90605643e-02 4.99895873e+00 5.99493837e+00
 3.00699947e+00 1.19708162e+01 2.01101983e+00 1.29667958e+01
 7.98689764e+00 6.99091800e+00 1.01504020e+00 1.39627754e+01
 2.29265921e+01 1.79466940e+01 1.49587551e+01 1.09748365e+01
 8.98287727e+00 1.89426736e+01 9.97885691e+00 1.59547347e+01
 2.09346329e+01 1.99386532e+01 1.69507143e+01 2.39225718e+01
 2.59145311e+01 2.69105107e+01 2.49185514e+01 3.18904089e+01
 2.19306125e+01 3.38823681e+01 2.79064903e+01 3.58743274e+01
 3.08944292e+01 4.18502052e+01 4.08542256e+01 5.08140219e+01
 3.68703070e+01 2.98984496e+01 2.89024700e+01 3.28863885e+01
 3.78662867e+01 3.48783478e+01 4.38421645e+01 3.98582459e+01
 4.28461848e+01]


[3.99291346e+00 2.38736552e-02 5.97743336e+00 3.00065351e+00
 2.00839356e+00 4.98517341e+00 1.01613361e+00 6.96969331e+00
 1.19309931e+01 7.96195326e+00 1.39155130e+01 1.29232530e+01
 8.95421321e+00 1.09387331e+01 1.78845528e+01 1.59000329e+01
 9.9464731

In [72]:
for col in numerical_columns:
    print(train_impute_DF[col].unique())

[1 0]
[0 1]
[0 1]
[0 1]
[1 0]
[1 0]
[0 1]
[4.00297910e+00 1.90605643e-02 4.99895873e+00 5.99493837e+00
 3.00699947e+00 1.19708162e+01 2.01101983e+00 1.29667958e+01
 7.98689764e+00 6.99091800e+00 1.01504020e+00 1.39627754e+01
 2.29265921e+01 1.79466940e+01 1.49587551e+01 1.09748365e+01
 8.98287727e+00 1.89426736e+01 9.97885691e+00 1.59547347e+01
 2.09346329e+01 1.99386532e+01 1.69507143e+01 2.39225718e+01
 2.59145311e+01 2.69105107e+01 2.49185514e+01 3.18904089e+01
 2.19306125e+01 3.38823681e+01 2.79064903e+01 3.58743274e+01
 3.08944292e+01 4.18502052e+01 4.08542256e+01 5.08140219e+01
 3.68703070e+01 2.98984496e+01 2.89024700e+01 3.28863885e+01
 3.78662867e+01 3.48783478e+01 4.38421645e+01 3.98582459e+01
 4.28461848e+01]
[3.99291346e+00 2.38736552e-02 5.97743336e+00 3.00065351e+00
 2.00839356e+00 4.98517341e+00 1.01613361e+00 6.96969331e+00
 1.19309931e+01 7.96195326e+00 1.39155130e+01 1.29232530e+01
 8.95421321e+00 1.09387331e+01 1.78845528e+01 1.59000329e+01
 9.94647316e+00 1.98690727

In [73]:
# categorical data에 합치기
train4_DF[numerical_columns] = train_impute_DF[numerical_columns]
test4_DF[numerical_columns] = test_impute_DF[numerical_columns]

In [74]:
# 잘 채워졌는지 확인 함 하고
for col in test4_DF.columns:
    print(test4_DF[col].unique())

['TRYBLT' 'TRDQAZ' 'TRCMWS' 'TRJXFG' 'TRXQMD' 'TRZKPL' 'TRVNRY']
['만35-37세' '만18-34세' '만40-42세' '만38-39세' '만43-44세' '만45-50세' nan]
['IVF' 'ICSI' nan 'IUI' 'ICSI / BLASTOCYST ' 'IVF:ICSI' 'ICSI:ICSI'
 'IVF / BLASTOCYST' 'ICSI / AH' 'ICSI:IVF' 'Generic DI' 'IVF:IVF'
 'IVF:Unknown' 'IVF / AH' 'IVF:Unknown:Unknown:Unknown' 'ICSI:Unknown'
 'IVI' 'ICI' 'ICSI / BLASTOCYST:IVF / BLASTOCYST' 'IUI:ICI']
[1 0]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
['현재 시술용' nan '배아 저장용' '배아 저장용, 현재 시술용' '기증용, 현재 시술용' '난자 저장용' '기증용'
 '기증용, 배아 저장용' '기증용, 난자 저장용' '기증용, 배아 저장용, 현재 시술용' '난자 저장용, 현재 시술용'
 '난자 저장용, 배아 저장용']
['1회' '0회' '3회' '5회' '2회' '4회' '6회 이상']
['1회' '0회' '3회' '5회' '2회' '4회' '6회 이상']
['0회' '2회' '1회' '3회' '5회' '4회' '6회 이상']
['0회' '1회' '2회' '3회' '4회' '5회' '6회 이상']
['0회' '1회' '2회' '4회' '3회']
[ 7.    0.    5.    2.   10.   14.   12.    1.    6.    4.    8.    5.5
  9.    3.   22.    3.25 11.   18.   16.    2.75 19.   15.   21.   13.
 24.    6.25 20.   30.   17.   27.   32.   25.   39.   31.   42.    4.5
 23

In [ ]:
# categorical np.nan -> "mssing"
for col in train4_DF.columns:
    if train4_DF[col].dtype == object:
        train4_DF[col] = train4_DF[col].fillna("missing")

for col in test4_DF.columns:
    if test4_DF[col].dtype == object:
        test4_DF[col] = test4_DF[col].fillna("missing")

# csv파일로 저장
train4_DF.to_csv('data/train4_updated.csv')
test4_DF.to_csv('data/test4_updated.csv')